In [48]:
import pandas as pd
import numpy as np
import os
import sys

utils_path = os.path.abspath('../../utils')
sys.path.insert(0, utils_path)

from teeth_utils import normalise_teeth_distances

In [49]:
FUTURE_BRACE_START_INDEX = 23
FUTURE_BRACE_END_INDEX = 41

In [50]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  

In [51]:
MERIDIAN_PATH = "../../data/life_data/life_sheet_final.csv"
TEETH_UP_PATH = "../../data/life_data/teeth_up_tracked_data.json"
TEETH_DOWN_PATH = "../../data/life_data/teeth_down_tracked_data.json"

In [52]:
teeth_up = pd.read_json(TEETH_UP_PATH)
teeth_down = pd.read_json(TEETH_DOWN_PATH)
teeth_df = pd.concat([teeth_up, teeth_down])
teeth_df = teeth_df.reset_index(drop=True)

In [53]:
fn = lambda v: {'toothId': v['toothId'], 'toothPositions': normalise_teeth_distances(v['toothPositions'])}
teeth_df["data"] = teeth_df["data"].apply(fn)

In [54]:
# set notgrownout teeth positions to 0
fn = lambda data: {"toothId": data["toothId"], "toothPositions": [ 0.0 for v in data["toothPositions"]]}
teeth_df["data"][0] = fn(teeth_df["data"][0])
teeth_df["data"][15] = fn(teeth_df["data"][15])
teeth_df["data"][16] = fn(teeth_df["data"][16])
teeth_df["data"][31] = fn(teeth_df["data"][31])

In [55]:
# convert string teeth indices to ints
meridian_df = pd.read_csv(MERIDIAN_PATH)
meridian_df.dropna(subset=["Tooth number", "Domain"], inplace=True)
fn = lambda s: [int(s) for s in s.split("-")]
meridian_df["Tooth number"] = meridian_df["Tooth number"].apply(fn)

In [56]:
# create the training dataset
predict_df = pd.DataFrame() 

# column represents data based on one of the multiple braces
# row is all teeth positions associated with a domain and the value of this domain 

# loop through meridian rows
for row_index, row in meridian_df.iterrows():    
    # get the meridian indexes of the relevant teeth
    teeth_ids = row["Tooth number"]
    # match relevant teeth against all teeth
    mask = teeth_df["data"].apply(lambda x: x['toothId'] in teeth_ids)
    associated_teeth = teeth_df["data"][mask]
    
    xs = []
    
    for brace_index in range(FUTURE_BRACE_START_INDEX, FUTURE_BRACE_END_INDEX):
        x = np.full((len(associated_teeth), 2), [-1.0, -1.0])
        for tooth_index, tooth in enumerate(associated_teeth):
             x[tooth_index] = tooth["toothPositions"][brace_index]
        xs.append(x)
        
    predict_df = predict_df.append(pd.Series(xs), ignore_index=True)

In [57]:
predict_df.shape

(98, 18)

In [58]:
out_dir = os.path.abspath('./output')

if not os.path.exists(out_dir):
   os.makedirs(out_dir)

out_path = os.path.join(out_dir, "predict_df.pickle")
predict_df.to_pickle(out_path)